In [109]:
# Josue Castellanos (2024)
# This script is for interferogram analysis as part of the PX Beamlines LDRD Project
import os
from glob import glob
import os
from PIL import Image
import pandas as pd
from io import StringIO
import numpy as np


# carpenmentalize the process to work with a certain data. 

# put images together as a motor coordinate, could be negative

# 3 or 4th data file in txt have coordinates

# Get coordinates from txt, associate the coordinates to KCube DC 2 Goal, Actual (Actual is the best alignment)

# Rotate - left side is west , right side is east, 90 degree rotation -90 or +90

# X motor is negative

# what direction is it scanning?

# radius coparison 

# Stitch - actual distance and camera, effective pixel size , subtract one image from another to see the difference

# Where are we on the surface, scan 601 , lets pass into the code the cropped rotated image that has the coordinate on the file name so we can reference that to where it is in space


## Functions

In [110]:
def GlobFileNames(dir_path):
    # if not os.path.isfile(dir_path):
    #     print(f"File not found: {dir_path}")
    #     return
    file_list = glob(dir_path)  # Sort list
    file_list.sort()
    return file_list

def ExtractData(fp):
    # Read the file content
    with open(fp, 'r') as file:
        lines = file.readlines()

    # Find the line where DATA starts
    for i, line in enumerate(lines):
        if 'DATA' in line:
            data_start_idx = i + 1
            break
    return data_start_idx, lines

def ReadData2DF(start_index, lines):
    # Read the data into a pandas DataFrame
    data_lines = lines[start_index:]
    data_str = ''.join(data_lines)
    
    # Convert the data into a DataFrame
    data = StringIO(data_str)
    df = pd.read_csv(data, sep='\t')
    return df

def CleanData(dict, column):
    df = dict.iloc[:,column - 1].tolist()
    return df

def ModifyFilename(file, series):
    basename = os.path.basename(file)
    # Remove a specific part of the string
    modified_basename = basename.replace(f'Image Single Motor Scan 000{series} 2D Image ', '')
    return modified_basename

def CropRotateImagesToRegion(series, file, startXpixel, endXpixel, startYpixel, endYpixel, coordinate, save_directory):
    # Open an image file
    with Image.open(file) as img:
        # Crop the image
        cropped_img = img.crop((startXpixel, startYpixel, endXpixel, endYpixel))
        # Rotate the image 90 degrees, Pillow 90 default is counterclockwise so -90 is needed for clockwise
        rotated_img = cropped_img.rotate(-90, expand=True)
        # Modify the new file name of image
        mod_name = ModifyFilename(file, series)
        # Save the cropped image
        cropped_file = os.path.join(save_directory, f'{series}_{coordinate}mm_{mod_name}')
        rotated_img.save(cropped_file)
        print(f"Image saved as: {cropped_file}")
        return cropped_file
    
def CropImages(series, files, startXpixel, endXpixel, startYpixel, endYpixel, coords, save_directory):
    cropped_files = []
    for file, coordinate in zip(files, coords):
        cropped_file = CropRotateImagesToRegion(series, file, startXpixel, endXpixel, startYpixel, endYpixel, np.round(coordinate, decimals=3), save_directory)
        if cropped_file:
            cropped_files.append(cropped_file)
    return cropped_files

def Subtract2Images(image1_path, image2_path, index1, index2, output_path):
    # Open the first image
    with Image.open(image1_path) as img1:
        img1_array = np.array(img1)
    
    # Open the second image
    with Image.open(image2_path) as img2:
        img2_array = np.array(img2)
    
    # Check both images have the same dimensions
    if img1_array.shape != img2_array.shape:
        raise ValueError("The images must have the same dimensions")

    # Subtract the two images
    subtracted_array = img1_array - img2_array
    # Clip the values to be in the range [0, 255]
    subtracted_array = np.clip(subtracted_array, 0, 255)
    # Convert the result back to a PIL image
    subtracted_image = Image.fromarray(subtracted_array.astype(np.uint8))
    # Save the resulting image to direcotry
    sub_file = os.path.join(output_path, f"Subtracted_{index2}_From_{index1}.Png")
    subtracted_image.save(sub_file, format='PNG')
    print(f'Subtracted image saved as: {output_path}')

def ReadModifiedImages(dirNum):
    # Read the cropped images from disk and return them as a list
    cropped_files = [os.path.join(dirNum, f) for f in os.listdir(dirNum) if "Crop_" in f]
    return cropped_files

## Demonstration of code analysis
**Glob all files**

In [111]:
# Below is testing the functionality
# Replace with the desired directory number 000599, 000600, 000601...etc
file_index = 652
date = 240709

# Replace with the actual path to the text file
basepath = f'/Volumes/SanDisk'  # My sandisk drive
dir_path = os.path.join(basepath, f'ASPIRES/{date}/Image Single Motor Scan 000{file_index} Images/*.Png')
header_path = os.path.join(basepath, f'ASPIRES/{date}/Image Single Motor Scan 000{file_index}.txt')

file_names = GlobFileNames(dir_path)

# Check if files are correct
for f in file_names:
    print(f)

/Volumes/SanDisk/ASPIRES/240709/Image Single Motor Scan 000652 Images/Image Single Motor Scan 000652 2D Image 001.Png
/Volumes/SanDisk/ASPIRES/240709/Image Single Motor Scan 000652 Images/Image Single Motor Scan 000652 2D Image 002.Png
/Volumes/SanDisk/ASPIRES/240709/Image Single Motor Scan 000652 Images/Image Single Motor Scan 000652 2D Image 003.Png
/Volumes/SanDisk/ASPIRES/240709/Image Single Motor Scan 000652 Images/Image Single Motor Scan 000652 2D Image 004.Png
/Volumes/SanDisk/ASPIRES/240709/Image Single Motor Scan 000652 Images/Image Single Motor Scan 000652 2D Image 005.Png
/Volumes/SanDisk/ASPIRES/240709/Image Single Motor Scan 000652 Images/Image Single Motor Scan 000652 2D Image 006.Png
/Volumes/SanDisk/ASPIRES/240709/Image Single Motor Scan 000652 Images/Image Single Motor Scan 000652 2D Image 007.Png
/Volumes/SanDisk/ASPIRES/240709/Image Single Motor Scan 000652 Images/Image Single Motor Scan 000652 2D Image 008.Png
/Volumes/SanDisk/ASPIRES/240709/Image Single Motor Scan 

In [112]:
# Extract All lines in txt file and provide a start index 
start_index, lines = ExtractData(header_path)
data = ReadData2DF(start_index, lines)

column = 4    # Column in dataframe
coordinates = CleanData(data, column)
coordinates

[0.0,
 0.2,
 0.40000001,
 0.60000002,
 0.80000001,
 1.0,
 1.20000005,
 1.39999998,
 1.60000002,
 1.79999995,
 2.0,
 2.20000005,
 2.4000001,
 2.5999999,
 2.79999995,
 3.0,
 3.20000005,
 3.4000001,
 3.5999999,
 3.79999995,
 4.0,
 4.19999981,
 4.4000001,
 4.5999999,
 4.80000019,
 5.0,
 5.19999981,
 5.4000001,
 5.5999999,
 5.80000019,
 6.0,
 6.19999981,
 6.4000001,
 6.5999999,
 6.80000019,
 7.0,
 7.19999981,
 7.4000001,
 7.5999999,
 7.80000019,
 8.0,
 8.19999981,
 8.39999962,
 8.60000038,
 8.80000019,
 9.0,
 9.19999981,
 9.39999962,
 9.60000038,
 9.80000019,
 10.0,
 10.19999981,
 10.39999962,
 10.60000038,
 10.80000019,
 11.0,
 11.19999981,
 11.39999962,
 11.60000038,
 11.80000019,
 12.0,
 12.19999981,
 12.39999962,
 12.60000038,
 12.80000019,
 13.0,
 13.19999981,
 13.39999962,
 13.60000038,
 13.80000019,
 14.0,
 14.19999981,
 14.39999962,
 14.60000038,
 14.80000019,
 15.0,
 15.19999981,
 15.39999962,
 15.60000038,
 15.80000019,
 16.0,
 16.20000076,
 16.39999962,
 16.60000038,
 16.79999924

In [113]:
startXpixel = 780
startYpixel = 10
endXpixel = 2500 + startXpixel
endYpixel = 1400 + startYpixel

save_directory = os.path.join(basepath, 'ASPIRES', f'Crop_Rot_000{file_index}')
new_folder_path = os.makedirs(save_directory, exist_ok=True)

cropped_files = CropImages(file_index, file_names, startXpixel, endXpixel, startYpixel, endYpixel, coordinates, save_directory)


None
Image saved as: /Volumes/SanDisk/ASPIRES/Crop_Rot_000652/652_0.0mm_001.Png
Image saved as: /Volumes/SanDisk/ASPIRES/Crop_Rot_000652/652_0.2mm_002.Png
Image saved as: /Volumes/SanDisk/ASPIRES/Crop_Rot_000652/652_0.4mm_003.Png
Image saved as: /Volumes/SanDisk/ASPIRES/Crop_Rot_000652/652_0.6mm_004.Png
Image saved as: /Volumes/SanDisk/ASPIRES/Crop_Rot_000652/652_0.8mm_005.Png
Image saved as: /Volumes/SanDisk/ASPIRES/Crop_Rot_000652/652_1.0mm_006.Png
Image saved as: /Volumes/SanDisk/ASPIRES/Crop_Rot_000652/652_1.2mm_007.Png
Image saved as: /Volumes/SanDisk/ASPIRES/Crop_Rot_000652/652_1.4mm_008.Png
Image saved as: /Volumes/SanDisk/ASPIRES/Crop_Rot_000652/652_1.6mm_009.Png
Image saved as: /Volumes/SanDisk/ASPIRES/Crop_Rot_000652/652_1.8mm_010.Png
Image saved as: /Volumes/SanDisk/ASPIRES/Crop_Rot_000652/652_2.0mm_011.Png
Image saved as: /Volumes/SanDisk/ASPIRES/Crop_Rot_000652/652_2.2mm_012.Png
Image saved as: /Volumes/SanDisk/ASPIRES/Crop_Rot_000652/652_2.4mm_013.Png
Image saved as: /Vol

In [114]:
# img1 = 599
# img2 = 600
# img1_path = f'/Beamline Controls/BCS Setup Data/{date}/Image Single Motor Scan 000{img1} Images'
# img2_path = f'/Beamline Controls/BCS Setup Data/{date}/Image Single Motor Scan 000{img2} Images'
# save_path = f'/Beamline Controls/BCS Setup Data/{date}/Subtracted Images'

# images1 = ReadModifiedImages(img1_path)
# images2 = ReadModifiedImages(img2_path)

# Subtract2Images(images1[0], images2[0], img2, img1, save_path)

**TESTING BELOW**